# aiida-aurora Plugin Test

In [1]:
import aiida
from aiida import load_profile
load_profile('aurora')
from aiida.orm import load_node, load_code, load_computer, load_group
from aiida.engine import submit

In [2]:
import ipywidgets as ipw
from IPython.display import display
import numpy as np
import pandas as pd
from collections import OrderedDict
import json, yaml, datetime
import aurora
import aiida_aurora
import aiida_aurora.data, aiida_aurora.parsers
import aurora.schemas.tomato_0p1
# from schemas.data_schemas import BatterySpecs, BatteryComposition, BatteryCapacity, BatteryMetadata, BatterySample, BatterySpecsJsonTypes, BatterySampleJsonTypes
from aurora.schemas.data_schemas import BatterySample as BatterySampleSchema, BatteryState as BatteryStateSchema

import matplotlib.pyplot as plt
%matplotlib widget

from aurora.interface.main import MainPanel

In [3]:
w_main = MainPanel()
display(w_main)

MainPanel(children=(HTML(value='<h2>Aurora</h2>'), Accordion(children=(Tab(children=(SampleFromId(children=(HT…

## TESTS

#### 1A. Store data from GUI

In [4]:
sample = w_main.selected_battery_sample
method = w_main.selected_cycling_protocol

In [16]:
sample.dict()

{'manufacturer': 'Maxell',
 'composition': {'description': 'LiMgO2-based'},
 'form_factor': '2016',
 'capacity': {'nominal': 60.0, 'actual': None, 'units': 'mAh'},
 'battery_id': 600,
 'metadata': {'name': '600_Maxell_20220320',
  'creation_datetime': Timestamp('2022-03-20 01:20:40+0000', tz='UTC'),
  'creation_process': 'Lorem ipsum dolor sit amet, consectetur adipiscing elit. Sed mollis arcu ac risus vestibulum, ac commodo nunc euismod. Vivamus suscipit eros nec justo faucibus, et eleifend diam eleifend.'}}

In [6]:
BatterySampleData = aiida.plugins.DataFactory('aurora.batterysample')
CyclingSpecsData = aiida.plugins.DataFactory('aurora.cyclingspecs')

In [7]:
sample_node = BatterySample(sample.dict())#, label='cuiao')
sample_node.store()

<BatterySample: uuid: 0d0ce68b-447a-4103-a122-93e948370fc6 (pk: 2)
{'capacity': {'units': 'mAh', 'actual': None, 'nominal': 60.0}, 'metadata': {'name': '600_Maxell_20220320', 'creation_process': 'Lorem ipsum dolor sit amet, consectetur adipiscing elit. Sed mollis arcu ac risus vestibulum, ac commodo nunc euismod. Vivamus suscipit eros nec justo faucibus, et eleifend diam eleifend.', 'creation_datetime': '2022-03-20T01:20:40+00:00'}, 'battery_id': 600, 'composition': {'description': 'LiMgO2-based'}, 'form_factor': '2016', 'manufacturer': 'Maxell'}>

In [9]:
method_node = CyclingSpecsData(method.dict())
method_node.store()

<CyclingSpecs: uuid: d427abb1-03c6-4a62-8118-4850cb6011bd (pk: 3)
{'method': [{'name': 'OCV_1', 'device': 'worker', 'technique': 'open_circuit_voltage', 'parameters': {'time': {'label': 'Time:', 'units': 's', 'value': 60.0, 'required': True, 'description': 'The length of the OCV step', 'default_value': 0.0}, 'E_range': {'label': 'E range', 'units': '', 'value': None, 'required': False, 'description': '', 'default_value': 'auto'}, 'I_range': {'label': 'I range', 'units': '', 'value': None, 'required': False, 'description': '', 'default_value': 'keep'}, 'record_every_dE': {'label': 'Record every $dE$:', 'units': 'V', 'value': None, 'required': False, 'description': 'Record a datapoint at prescribed voltage spacing', 'default_value': 0.005}, 'record_every_dt': {'label': 'Record every $dt$:', 'units': 's', 'value': None, 'required': False, 'description': 'Record a datapoint at prescribed time spacing', 'default_value': 30.0}}, 'short_name': 'OCV', 'description': 'Open circuit voltage'}]}>

#### 1B. Dummy Data

In [3]:
sample = BatterySampleSchema(
    manufacturer = 'fake',
    composition = dict(description='C|E|A'),
    form_factor = 'DUMB',
    capacity = dict(nominal=1.0, units='Ah'),
    battery_id = 666,
    metadata = dict(
        name = 'fake_sample',
        creation_datetime = datetime.datetime.now(tz=datetime.timezone.utc),
        creation_process = 'This is a fake battery for testing purposes.'
    ))

In [ ]:
sample_node = aiida_aurora.data.BatterySampleData(sample.dict())
sample_node.store()

In [8]:
step = aurora.schemas.cycling.Dummy(technique="random")
step.parameters['time'].value = 30.0
step.parameters['delay'].value = 5.0
method = aurora.schemas.cycling.ElectroChemSequence(method=[step])

In [7]:
step = aurora.schemas.cycling.Dummy(technique="sequential")
step.parameters['time'].value = 1800
step.parameters['delay'].value = 5
step.name = "Dummy-Sequential"
method = aurora.schemas.cycling.ElectroChemSequence(method=[step])

method_node = aiida_aurora.data.CyclingSpecsData(method.dict())
method_node.label = 'dummy-sequential'
method_node.store()

<CyclingSpecs: uuid: 78e0e63a-974b-47a3-8c7a-b028f7e69a99 (pk: 134)
{'method': [{'name': 'Dummy-Sequential', 'device': 'worker', 'technique': 'sequential', 'parameters': {'time': {'label': 'Time:', 'units': 's', 'value': 1800.0, 'required': True, 'description': '', 'default_value': 10.0}, 'delay': {'label': 'Delay:', 'units': 's', 'value': 5.0, 'required': True, 'description': '', 'default_value': 1.0}}, 'short_name': 'DUMMY', 'description': 'Dummy Random worker'}]}>

In [7]:
tomato_sett = aurora.schemas.tomato_0p2.Tomato(
    snapshot = {'frequency': 10, 'prefix': 'snapshot'})
tomato_sett_node = aiida_aurora.data.TomatoSettingsData(tomato_sett.dict())
tomato_sett_node.label = 'dummy-snapshot-10'
tomato_sett_node.store()

<TomatoSettingsData: uuid: 0c28e406-22b7-4588-9de9-c4399d459e62 (pk: 139)
{'output': {'path': None, 'prefix': None}, 'snapshot': {'path': None, 'prefix': 'snapshot', 'frequency': 10}, 'verbosity': 'WARNING', 'unlock_when_done': False}>

#### 2. Prepare CalcJob

In [4]:
# sample_node = load_node(2)  # '600_Maxell_20220320'
# method_node = load_node(3)  # 'ocv-test'

sample_node = load_node(31)  # 'fake_sample'
# method_node = load_node(32)  # 'dummy-random'
method_node = load_node(134)  # 'dummy-sequential'

In [5]:
sample_node.get_dict()

{'capacity': {'units': 'Ah', 'actual': None, 'nominal': 1.0},
 'metadata': {'name': 'fake_sample',
  'creation_process': 'This is a fake battery for testing purposes.',
  'creation_datetime': '2022-07-04T14:09:46.338584+00:00'},
 'battery_id': 666,
 'composition': {'description': 'C|E|A'},
 'form_factor': 'DUMB',
 'manufacturer': 'fake'}

In [8]:
method_node.get_dict()

{'method': [{'name': 'Dummy-Sequential',
   'device': 'worker',
   'technique': 'sequential',
   'parameters': {'time': {'label': 'Time:',
     'units': 's',
     'value': 1800.0,
     'required': True,
     'description': '',
     'default_value': 10.0},
    'delay': {'label': 'Delay:',
     'units': 's',
     'value': 5.0,
     'required': True,
     'description': '',
     'default_value': 1.0}},
   'short_name': 'DUMMY',
   'description': 'Dummy Random worker'}]}

In [9]:
BatteryCyclerExperiment = aiida.plugins.CalculationFactory('aurora.cycler')
code = load_code('ketchup@localhost')

In [10]:
b = BatteryCyclerExperiment.get_builder()

b.battery_sample = sample_node
b.code = code
b.technique = method_node
# b.metadata.dry_run = True

In [ ]:
# exp = submit(b)

#### 3. Test Calc Monitor

In [11]:
sample_node = load_node(31)  # 'fake_sample'
method_node = load_node(134)  # 'dummy-sequential'
settings_node = load_node(139)  # 'dummy-snapshot-10'

In [12]:
settings_node.get_dict()

{'output': {'path': None, 'prefix': None},
 'snapshot': {'path': None, 'prefix': 'snapshot', 'frequency': 10},
 'verbosity': 'WARNING',
 'unlock_when_done': False}

In [13]:
DummyCycler = aiida.plugins.CalculationFactory('aurora.cycler')
code = load_code('ketchup@localhost')

In [14]:
b = DummyCycler.get_builder()

b.battery_sample = sample_node
b.code = code
b.technique = method_node
b.control_settings = settings_node
# b.metadata.dry_run = True

In [8]:
exp = submit(b)

08/09/2022 04:51:47 PM <39418> aiida.engine.persistence: [DEBUG] Persisting process<191>
08/09/2022 04:51:47 PM <39418> aiida.engine.persistence: [DEBUG] Persisting process<191>


In [16]:
ToymodelMonitor = aiida.plugins.DataFactory('calcmonitor.monitor.tomatodummy')
monitor_protocol = ToymodelMonitor(dict={
    'sources': {
        'output': {'filepath': 'snapshot.json', 'refresh_rate': 10},
    },
    'options': {},
    'retrieve': ['snapshot.json', 'snapshot.zip'],
})
MonitorCalcjob = aiida.plugins.CalculationFactory('calcmonitor.calcjob_monitor')
monitor_builder = MonitorCalcjob.get_builder()
monitor_builder.code = load_code('monitor@localhost-direct')
monitor_builder.monitor_protocols = {'monitor1': monitor_protocol}

In [10]:
# exp = load_node(151)

In [11]:
monitor_builder.monitor_folder = exp.outputs.remote_folder
calcjob_node = submit(monitor_builder)

08/09/2022 04:52:11 PM <39418> aiida.engine.persistence: [DEBUG] Persisting process<194>
08/09/2022 04:52:11 PM <39418> aiida.engine.persistence: [DEBUG] Persisting process<194>


---
#### A. convert schemas to tomato-compatible ones

In [20]:
sample = w_main.selected_battery_sample
method = w_main.selected_cycling_protocol

In [21]:
sample.dict()

{'manufacturer': 'Conrad energy',
 'composition': {'description': 'Li-based'},
 'form_factor': '2032',
 'capacity': {'nominal': 45.0, 'actual': None, 'units': 'mAh'},
 'battery_id': 2,
 'metadata': {'name': 'commercial-2',
  'creation_datetime': Timestamp('2022-06-28 05:59:00+0000', tz='UTC'),
  'creation_process': 'Bought in a shop.'}}

In [75]:
print(yaml.dump(aurora.schemas.tomato_0p1.sample.convert_batterysample_to_sample(sample).dict()))

capacity: 0.045
name: commercial-2



In [22]:
method.dict()

{'method': [{'device': 'worker',
   'technique': 'open_circuit_voltage',
   'short_name': 'OCV',
   'name': 'OCV_1',
   'description': 'Open circuit voltage',
   'parameters': {'time': {'label': 'Time:',
     'description': 'The length of the OCV step',
     'units': 's',
     'value': 22.0,
     'default_value': 0.0,
     'required': True},
    'record_every_dt': {'label': 'Record every $dt$:',
     'description': 'Record a datapoint at prescribed time spacing',
     'units': 's',
     'value': 30.0,
     'default_value': 30.0,
     'required': True},
    'record_every_dE': {'label': 'Record every $dE$:',
     'description': 'Record a datapoint at prescribed voltage spacing',
     'units': 'V',
     'value': 0.005,
     'default_value': 0.005,
     'required': True},
    'I_range': {'label': 'I range',
     'description': '',
     'units': '',
     'value': 'keep',
     'default_value': 'keep',
     'required': True},
    'E_range': {'label': 'E range',
     'description': '',
     'u

In [23]:
print(yaml.dump([elem.dict() for elem in aurora.schemas.tomato_0p1.method.convert_electrochemsequence_to_method_list(method)]))

- E_range: auto
  I_range: keep
  device: worker
  record_every_dE: 0.005
  record_every_dt: 30.0
  technique: open_circuit_voltage
  time: 22.0
- E_range: auto
  I_range: keep
  current: 0.0
  device: worker
  exit_on_limit: false
  is_delta: false
  n_cycles: 0
  record_every_dE: 0.001
  record_every_dt: 30.0
  technique: constant_current
  time: 33.0
- device: worker
  technique: open_circuit_voltage



In [24]:
t = aurora.schemas.tomato_0p1.tomato.Tomato(**{'output': {'prefix': 'ciao'}})
print(yaml.dump(t.dict()))

output:
  path: null
  prefix: ciao
unlock_when_done: false
verbosity: INFO



In [25]:
a = aurora.schemas.tomato_0p1.TomatoPayload(
    version = "0.1",
    sample = aurora.schemas.tomato_0p1.sample.convert_batterysample_to_sample(sample.dict()),
    method = aurora.schemas.tomato_0p1.method.convert_electrochemsequence_to_method_list(method.dict()),
    tomato = aurora.schemas.tomato_0p1.tomato.Tomato(output={'prefix': 'ciao'})
)
a

TomatoPayload(version='0.1', tomato=Tomato(unlock_when_done=False, verbosity='INFO', output=Output(path=None, prefix='ciao')), sample=Sample(name='commercial-2', capacity=0.045), method=[Method(device='worker', technique='open_circuit_voltage', record_every_dt=30.0, record_every_dE=0.005, I_range='keep', E_range='auto', time=22.0), Method(device='worker', technique='constant_current', n_cycles=0, I_range='keep', exit_on_limit=False, current=0.0, record_every_dt=30.0, record_every_dE=0.001, E_range='auto', time=33.0, is_delta=False), Method(device='worker', technique='open_circuit_voltage')])

In [27]:
print(yaml.dump(a.dict()))

method:
- E_range: auto
  I_range: keep
  device: worker
  record_every_dE: 0.005
  record_every_dt: 30.0
  technique: open_circuit_voltage
  time: 22.0
- E_range: auto
  I_range: keep
  current: 0.0
  device: worker
  exit_on_limit: false
  is_delta: false
  n_cycles: 0
  record_every_dE: 0.001
  record_every_dt: 30.0
  technique: constant_current
  time: 33.0
- device: worker
  technique: open_circuit_voltage
sample:
  capacity: 0.045
  name: commercial-2
tomato:
  output:
    prefix: ciao
  unlock_when_done: false
  verbosity: INFO
version: '0.1'



#### B. Parse output JSON file

In [40]:
with open('/home/lercole/src/Aurora/examples/plugin_test/results.38.json', 'r') as f:
    data = json.load(f)
print(data.keys())

dict_keys(['metadata', 'steps'])


In [41]:
n = aiida_aurora.parsers.TomatoParser.parse_tomato_results(data)

0 ['value']


In [42]:
print(yaml.dump(data['metadata']))

datagram_version: 4.1.0rc2
date: '2022-07-05 09:22:25'
input_schema:
  metadata:
    provenance:
      metadata:
        preset_provenance:
          type: tomato
      type: yadg preset
    timezone: localtime
    version: 4.1.1
  steps:
  - externaldate:
      mode: add
      using:
        file:
          match: uts
          path: /scratch/lercole/AIIDA_SCRATCH/aurora/tomato/0.1rc11.post0.dev1/Jobs/38/worker_status.json
          type: json
    input:
      contains: null
      encoding: UTF-8
      exclude: null
      files:
      - /scratch/lercole/AIIDA_SCRATCH/aurora/tomato/0.1rc11.post0.dev1/Jobs/38
      prefix: worker
      suffix: data.json
    parameters:
      filetype: tomato.json
    parser: dummy
    tag: worker
provenance:
  data:
    archive: /home/lercole/src/Aurora/examples/plugin_test/results.38.zip
    sha-1: 8d7f9c9cf95b3a922e715c9dfc338da75976c2c2
  yadg:
    command: /home/lercole/.virtualenvs/Aurora/bin/yadg preset -pa preset.38.json
      /scratch/lercole/AI

In [43]:
data['steps']

[{'metadata': {'tag': 'worker', 'parser': 'dummy'},
  'data': [{'uts': 1657012908.728301,
    'fn': '/scratch/lercole/AIIDA_SCRATCH/aurora/tomato/0.1rc11.post0.dev1/Jobs/38/worker_2022-07-05T092149.739427+0000_data.json',
    'raw': {'value': {'n': 0.09795002685345544, 's': 0.0, 'u': ' '}}},
   {'uts': 1657012908.728301,
    'fn': '/scratch/lercole/AIIDA_SCRATCH/aurora/tomato/0.1rc11.post0.dev1/Jobs/38/worker_2022-07-05T092149.739427+0000_data.json',
    'raw': {'value': {'n': 0.11345391168432484, 's': 0.0, 'u': ' '}}},
   {'uts': 1657012909.7284527,
    'fn': '/scratch/lercole/AIIDA_SCRATCH/aurora/tomato/0.1rc11.post0.dev1/Jobs/38/worker_2022-07-05T092149.739427+0000_data.json',
    'raw': {'value': {'n': 0.31839394507831287, 's': 0.0, 'u': ' '}}},
   {'uts': 1657012910.7283015,
    'fn': '/scratch/lercole/AIIDA_SCRATCH/aurora/tomato/0.1rc11.post0.dev1/Jobs/38/worker_2022-07-05T092150.743177+0000_data.json',
    'raw': {'value': {'n': 0.5768880525131215, 's': 0.0, 'u': ' '}}},
   {'ut

In [44]:
array_dic = {}
for imstep, mstep in enumerate(data['steps']):  # method step
    raw_qty_names = mstep['data'][0]['raw'].keys()
    print(imstep, list(raw_qty_names))
    for raw_qty_name in raw_qty_names:
        for identifier in ('n', 's', 'u'):
            array_dic[f'step{imstep}_{raw_qty_name}_{identifier}'] = np.array([step['raw'][raw_qty_name][identifier] for step in mstep['data']])
    array_dic[f'step{imstep}_uts'] = np.array([step['uts'] for step in mstep['data']])

0 ['value']


In [47]:
array_dic['step0_value_n'].shape

(38,)

In [114]:
n = aiida.orm.ArrayData()
for key, value in array_dic.items():
    print(key,value)
    n.set_array(key, value)
n.set_attribute_many(data['metadata'])

step0_value_n [0.98944914 0.1499606  0.30144479 0.44437608 0.93691806 0.69036869]
step0_value_s [0. 0. 0. 0. 0. 0.]
step0_value_u [' ' ' ' ' ' ' ' ' ' ' ']
step0_uts [1.65694972e+09 1.65694972e+09 1.65694973e+09 1.65694973e+09
 1.65694974e+09 1.65694974e+09]


In [115]:
n.attributes

{'array|step0_value_n': [6],
 'array|step0_value_s': [6],
 'array|step0_value_u': [6],
 'array|step0_uts': [6],
 'provenance': {'yadg': {'version': '4.1',
   'command': '/home/lercole/.virtualenvs/Aurora/bin/yadg preset -pa preset.36.json /scratch/lercole/AIIDA_SCRATCH/aurora/tomato/0.1rc11.post0.dev1/Jobs/36 /scratch/lercole/AIIDA_SCRATCH/aurora/e8/d3/f106-0e40-4631-8bb0-78e41988c338/results.json'},
  'data': {'sha-1': 'ebf91276f3ee629be0458621ab7a757e2d304c3c',
   'archive': '/scratch/lercole/AIIDA_SCRATCH/aurora/e8/d3/f106-0e40-4631-8bb0-78e41988c338/results.zip'}},
 'date': '2022-07-04 15:49:07',
 'input_schema': {'metadata': {'provenance': {'type': 'yadg preset',
    'metadata': {'preset_provenance': {'type': 'tomato'}}},
   'version': '4.1.1',
   'timezone': 'localtime'},
  'steps': [{'parser': 'dummy',
    'input': {'files': ['/scratch/lercole/AIIDA_SCRATCH/aurora/tomato/0.1rc11.post0.dev1/Jobs/36'],
     'prefix': 'worker',
     'suffix': 'data.json',
     'contains': None,
   